## Sentinel Data download & transfor expetriment 2

## Load libs

In [3]:
import os
from dotenv import load_dotenv

from datetime import date, timedelta
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

In [7]:
load_dotenv()

True

In [8]:
test = os.getenv("test")
print(test)

This is a testenv


## Authentication

In [ ]:
copernicus_user = os.getenv("copernicus_user") # copernicus User
copernicus_password = os.getenv("copernicus_password") # copernicus Password

Area of Interest 

In [9]:
ft = "POLYGON ((5.922901 49.910716, 5.925991 49.911822, 5.919296 49.914256, 5.910713 49.913039, 5.904362 49.912597, 5.898869 49.915583, 5.888054 49.912707, 5.880672 49.906845, 5.876553 49.905739, 5.871746 49.908947, 5.873978 49.911712, 5.870373 49.912486, 5.866425 49.910274, 5.860245 49.916357, 5.856125 49.914919, 5.853035 49.919122, 5.850117 49.922439, 5.84119 49.92255, 5.85458 49.911048, 5.865223 49.906181, 5.865395 49.904079, 5.85973 49.907177, 5.850975 49.91138, 5.845997 49.9095, 5.850117 49.906292, 5.843422 49.901535, 5.838444 49.895007, 5.8381 49.890913, 5.829517 49.88538, 5.834839 49.875198, 5.825226 49.868666, 5.827457 49.864681, 5.844109 49.871988, 5.850117 49.877743, 5.839645 49.881507, 5.841362 49.884052, 5.850975 49.880621, 5.859043 49.886819, 5.849773 49.89213, 5.857327 49.896999, 5.863678 49.894565, 5.875351 49.899543, 5.888226 49.901756, 5.891144 49.894786, 5.89492 49.896114, 5.89389 49.907619, 5.905392 49.904522, 5.897495 49.897663, 5.898525 49.895229, 5.901958 49.894786, 5.903503 49.898437, 5.91243 49.902199, 5.920155 49.899654, 5.924961 49.902309, 5.922901 49.910716))"

Which Satellite

In [10]:
data_collection = "SENTINEL-2"

dates

In [17]:
today = date.today()
today_string = today.strftime("%Y-%m-%d")
yesterday = today - timedelta(days = 30)
yesterday_string = yesterday.strftime("%Y-%m-%d")

# start_date = "2025-06-22"
# end_date = "2025-06-22"

# today_string = end_date
# yesterday_string = start_date

## Generate access token

In [13]:
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]

## Access API to get data

In [18]:
json_ = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and ContentDate/Start gt {yesterday_string}T00:00:00.000Z and ContentDate/Start lt {today_string}T00:00:00.000Z&$count=True&$top=1000"
).json()  
p = pd.DataFrame.from_dict(json_["value"]) # Fetch available dataset
if p.shape[0] > 0 : # If we get data back
    p["geometry"] = p["GeoFootprint"].apply(shape)

    # Convert pandas dataframe to Geopandas dataframe by setting up geometry
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry") 

    # Remove L1C dataset if not needed
    # productDF = productDF[~productDF["Name"].str.contains("L1C")] 
    print(f" total L2A tiles found {len(productDF)}")
    productDF["identifier"] = productDF["Name"].str.split(".").str[0]
    allfeat = len(productDF) 
    
    if allfeat == 0: # If L2A tiles are not available in current query
        print(f"No tiles found for {today_string}")
    else: # If L2A tiles are available in current query
        # download all tiles from server
        for index,feat in enumerate(productDF.iterfeatures()):
            try:
                # Create requests session 
                session = requests.Session()

                # Get access token based on username and password
                keycloak_token = get_keycloak(copernicus_user,copernicus_password)
                
                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects=False)

                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get(url, allow_redirects=False)
                
                print(feat["properties"]["Id"])
                
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"/data/{feat['properties']['identifier']}.zip", #location to save zip from copernicus 
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except:
                print("problem with server")
else : # If no tiles found for given date range and AOI
    print('no data found')

 total L2A tiles found 32
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server
problem with server


## 